In [1]:
from sfm.data.sci_data.SFMDecTokenizer import SFMDecTokenizer
import torch
import os

tokenizer_home = '/hai1/ds_dataset/llama2/llama-2-7b'

tokenizer = SFMDecTokenizer.from_pretrained(tokenizer_home)

/anaconda/envs/sfm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-11-16 07:39:39,651] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LlamaTokenizer'. 
The class this function is called from is 'SFMDecTokenizer'.
You are using the default legacy behaviour of the <class 'sfm.data.sci_data.SFMDecTokenizer.SFMDecTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [2]:
print(len(tokenizer))


34177


In [3]:
print(tokenizer("<protein></protein>").input_ids)
print(tokenizer("a drug for HIV is <mol>").input_ids)

[1, 32007, 32008]
[1, 263, 15721, 363, 379, 5667, 338, 32001]


In [4]:
tokenizer.batch_encode_plus(["<protein></protein>", "a drug for HIV is <mol>"], padding=True, return_tensors='pt').input_ids

tensor([[    1, 32007, 32008, 32000, 32000, 32000, 32000, 32000],
        [    1,   263, 15721,   363,   379,  5667,   338, 32001]])

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
ckpt_home = '/hai1/shufxi/scigpt/7b/stageB/global_step16599/'
ckpt_path = os.path.join(ckpt_home, 'layer_33-model_states.pt')
model_states = torch.load(ckpt_path, map_location='cpu')
all_keys = list(model_states.keys())
print(all_keys)

['dummy.weight', 'dummy.bias', 'norm.weight']


In [6]:
def show_ckpt(name, ckpt):
    for k, v in ckpt.items():
        if 'dummy' not in k:
            print(name, k, v.shape)

model = AutoModelForCausalLM.from_pretrained(tokenizer_home)
#model.resize_token_embeddings(len(tokenizer) + 1)

model_dict = model.state_dict()
ckpt_dict = {}
layer0 = torch.load(os.path.join(ckpt_home, "layer_00-model_states.pt"), map_location=torch.device("cpu"))
ckpt_dict['model.embed_tokens.weight'] = layer0['embed_tokens.weight']
show_ckpt('layer0', layer0)

for l in range(0, 32):
    l_index = str(l + 1).zfill(2)
    #print(f"../blob2/checkpoints/MetaLLM-7B-D_NODE4-D_PROC16-instruction-mt/global_step97077/layer_{l_index}-model_states.pt")
    layer = torch.load(os.path.join(ckpt_home, f"layer_{l_index}-model_states.pt"), map_location=torch.device("cpu"))
    show_ckpt(l_index, layer)
    for k in layer:
        if "dummy" in k or 'rotary_emb' in k:
            continue
        ckpt_dict[f"model.layers.{l}.{k}"] = layer[k]
layer = torch.load(os.path.join(ckpt_home, "layer_33-model_states.pt"), map_location=torch.device("cpu"))
show_ckpt(33, layer)
ckpt_dict["model.norm.weight"] = layer["norm.weight"]

layer = torch.load(os.path.join(ckpt_home, "layer_34-model_states.pt"), map_location=torch.device("cpu"))
show_ckpt(33, layer)
ckpt_dict["lm_head.weight"] = layer["lm_head.weight"]
model_dict.update(ckpt_dict)

model.resize_token_embeddings(len(tokenizer))
model.load_state_dict(model_dict)

# tokenizer.save_pretrained("/home/v-zequnliu/blob/v-zequnliu/llama/scigpt_7b_199")
# model.save_pretrained("/home/v-zequnliu/blob/v-zequnliu/llama/scigpt_7b_199")

Loading checkpoint shards: 100%|██████████| 2/2 [00:25<00:00, 12.95s/it]


layer0 embed_tokens.weight torch.Size([34177, 4096])
01 self_attn.q_proj.weight torch.Size([4096, 4096])
01 self_attn.k_proj.weight torch.Size([4096, 4096])
01 self_attn.v_proj.weight torch.Size([4096, 4096])
01 self_attn.o_proj.weight torch.Size([4096, 4096])
01 self_attn.rotary_emb.inv_freq torch.Size([64])
01 mlp.gate_proj.weight torch.Size([11008, 4096])
01 mlp.up_proj.weight torch.Size([11008, 4096])
01 mlp.down_proj.weight torch.Size([4096, 11008])
01 input_layernorm.weight torch.Size([4096])
01 post_attention_layernorm.weight torch.Size([4096])
02 self_attn.q_proj.weight torch.Size([4096, 4096])
02 self_attn.k_proj.weight torch.Size([4096, 4096])
02 self_attn.v_proj.weight torch.Size([4096, 4096])
02 self_attn.o_proj.weight torch.Size([4096, 4096])
02 self_attn.rotary_emb.inv_freq torch.Size([64])
02 mlp.gate_proj.weight torch.Size([11008, 4096])
02 mlp.up_proj.weight torch.Size([11008, 4096])
02 mlp.down_proj.weight torch.Size([4096, 11008])
02 input_layernorm.weight torch.Size

<All keys matched successfully>

In [7]:
print(l_index)

32


In [9]:
model = model.cuda()

In [9]:
tokenizer.tokenize("a drug for HIV is <mol>")

['▁a', '▁drug', '▁for', '▁H', 'IV', '▁is', '<mol>']

In [10]:
text = "a drug for HIV is <mol>"
input_ids = tokenizer(text, return_tensors="pt").input_ids.cuda()
print(input_ids)
#bad_words = [[i] for i in range(3, 32001)]
#bad_words.append([0])
output = model.generate(input_ids, do_sample=True, top_p=0.9, max_new_tokens=100, num_beams=5, num_return_sequences=1)
print(output)

tensor([[    1,   263, 15721,   363,   379,  5667,   338, 32001]],
       device='cuda:0')


tensor([[    1,   263, 15721,   363,   379,  5667,   338, 32001, 32124, 32124,
         32125, 32124, 32126, 32125, 32124, 32126, 32131, 32124, 32125, 32129,
         32128, 32126, 32140, 32127, 32124, 32131, 32125, 32124, 32123, 32130,
         32123, 32123, 32123, 32132, 32123, 32130, 32126, 32124, 32124, 32131,
         32127, 32124, 32140, 32125, 32128, 32126, 32124, 32140, 32125, 32124,
         32123, 32127, 32123, 32123, 32123, 32123, 32123, 32127, 32126, 32124,
         32125, 32129, 32128, 32126, 32131, 32139, 32127, 32123, 32130, 32123,
         32123, 32123, 32123, 32123, 32130, 32124, 32139, 32127, 32128, 32002,
             2]], device='cuda:0')


In [11]:
for t in output:
    print(tokenizer.decode(t, skip_special_tokens=True))

a drug for HIV is <mol> <m>C <m>C <m>( <m>C <m>) <m>( <m>C <m>) <m>N <m>C <m>( <m>= <m>O <m>) <m>[C@@H] <m>1 <m>C <m>N <m>( <m>C <m>c <m>2 <m>c <m>c <m>c <m>n <m>c <m>2 <m>) <m>C <m>C <m>N <m>1 <m>C <m>[C@@H] <m>( <m>O <m>) <m>C <m>[C@@H] <m>( <m>C <m>c <m>1 <m>c <m>c <m>c <m>c <m>c <m>1 <m>) <m>C <m>( <m>= <m>O <m>) <m>N <m>[C@H] <m>1 <m>c <m>2 <m>c <m>c <m>c <m>c <m>c <m>2 <m>C <m>[C@H] <m>1 <m>O </mol>


In [12]:
text = "The aspirin is represented as <mol>"
input_ids = tokenizer(text, return_tensors="pt").input_ids.cuda()
output = model.generate(input_ids, do_sample=True, top_p=0.9, max_new_tokens=200, num_beams=4, num_return_sequences=4)

for s in output:
    print(tokenizer.decode(s))

<s>The aspirin is represented as <mol> <m>C <m>C <m>( <m>= <m>O <m>) <m>O <m>c <m>1 <m>c <m>c <m>c <m>c <m>c <m>1 <m>C <m>( <m>= <m>O <m>) <m>[O-] <m>. <m>C <m>C <m>( <m>= <m>O <m>) <m>O <m>c <m>1 <m>c <m>c <m>c <m>c <m>c <m>1 <m>C <m>( <m>= <m>O <m>) <m>[O-] <m>. <m>[Ca+2] </mol> in the SMILES sequence: <mol> <m>C <m>C <m>( <m>= <m>O <m>) <m>O <m>c <m>1 <m>c <m>c <m>c <m>c <m>c <m>1 <m>C <m>( <m>= <m>O <m>) <m>[O-] <m>. <m>C <m>C <m>( <m>= <m>O <m>) <m>O <m>c <m>1 <m>c <m>c <m>c <m>c <m>c <m>1 <m>C <m>( <m>= <m>O <m>) <m>[O-] <m>. <m>[Ca+2] </mol> . Acetylsalicylic acid is a monocarboxylic acid that is the acetyl derivative of salicylic acid. It has a role as a non-steroidal anti-inflammatory drug, a non-narcotic analgesic, an antipyretic, an EC 1.14.99.1 (prostaglandin-endoperoxide synthase) inhibitor, an EC 1
<s>The aspirin is represented as <mol> <m>C <m>C <m>( <m>= <m>O <m>) <m>O <m>c <m>1 <m>c <m>c <m>c <m>c <m>c <m>1 <m>C <m>( <m>= <m>O <m>) <m>[O-] <m>. <m>C <m>C <m>( <m>= <m>O

In [13]:
text = "The SMILES of aspirin is <mol>"
input_ids = tokenizer(text, return_tensors="pt").input_ids.cuda()
output = model.generate(input_ids, do_sample=True, top_p=0.9, max_new_tokens=200, num_beams=4, num_return_sequences=4)

for s in output:
    print(tokenizer.decode(s))

<s>The SMILES of aspirin is <mol> <m>C <m>C <m>( <m>= <m>O <m>) <m>O <m>c <m>1 <m>c <m>c <m>c <m>c <m>c <m>1 <m>C <m>( <m>= <m>O <m>) <m>O </mol>  .</s>
<s>The SMILES of aspirin is <mol> <m>C <m>C <m>( <m>= <m>O <m>) <m>O <m>c <m>1 <m>c <m>c <m>c <m>c <m>c <m>1 <m>C <m>( <m>= <m>O <m>) <m>O </mol>  .</s>
<s>The SMILES of aspirin is <mol> <m>C <m>C <m>( <m>= <m>O <m>) <m>O <m>c <m>1 <m>c <m>c <m>c <m>c <m>c <m>1 <m>C <m>( <m>= <m>O <m>) <m>O </mol>  .</s>
<s>The SMILES of aspirin is <mol> <m>C <m>C <m>( <m>= <m>O <m>) <m>O <m>c <m>1 <m>c <m>c <m>c <m>c <m>c <m>1 <m>C <m>( <m>= <m>O <m>) <m>O </mol>  .</s>


In [14]:
text = "a protein with 4 helixes is <protein>"
input_ids = tokenizer(text, return_tensors="pt").input_ids.cuda()
#bad_words = [[i] for i in range(3, 32001)]
#bad_words.append([0])
output = model.generate(input_ids, do_sample=True, top_p=0.9, max_new_tokens=100, num_beams=5, num_return_sequences=1)

In [15]:
for t in output:
    print(tokenizer.decode(t, skip_special_tokens=True))

a protein with 4 helixes is <protein> <a>M <a>K <a>K <a>I <a>E <a>A <a>I <a>I <a>R <a>P <a>F <a>K <a>L <a>D <a>E <a>V <a>K <a>I <a>A <a>L <a>V <a>N <a>A <a>G <a>I <a>V <a>G <a>M <a>T <a>V <a>S <a>E <a>V <a>R <a>G <a>F <a>G <a>R <a>Q <a>K <a>G <a>Q <a>T <a>E <a>R <a>Y <a>R <a>G <a>S <a>E <a>Y <a>T <a>V <a>E <a>F <a>L <a>Q <a>K <a>L <a>K <a>L <a>E <a>I <a>V <a>V <a>E <a>D <a>A <a>Q <a>V <a>D <a>T <a>V <a>I <a>D <a>K <a>I <a>V <a>A <a>A <a>A <a>R <a>T <a>G <a>E <a>I <a>G <a>D <a>G <a>K <a>I <a>F <a>V <a>S <a>P <a>V <a>D <a>Q <a>T <a>I


In [16]:
text = "The treatment of SARS-CoV-2"
input_ids = tokenizer(text, return_tensors="pt").input_ids.cuda()
#bad_words = [[i] for i in range(3, 32001)]
#bad_words.append([0])
output = model.generate(input_ids, do_sample=False, max_new_tokens=200, num_beams=5, num_return_sequences=1)

KeyboardInterrupt: 

In [ ]:
for t in output:
    print(tokenizer.decode(t, skip_special_tokens=True))

The treatment of SARS-CoV-2 infection is mainly dependent on the use of antivirals that target the viral RNA-dependent RNA polymerase (RdRp) non-structural protein 13 (nsp13) and the main protease (M <sup>pro </sup>) to inhibit viral replication and transcription, respectively <sup>1,2 </sup>. However, viral variants that harbour mutations in these proteins can lead to drug resistance <sup>3-5 </sup>. An alternative viral target is the nsp16-nsp10 helicase-protease, which is essential for viral replication <sup>6,7 </sup>. Here we report the discovery and characterization of GC376, a potent inhibitor of SARS-CoV-2 nsp16-nsp10. The crystal structure of nsp16-


In [ ]:
text = "The <mol> CC(=O)Oc1ccccc1C(=O)O </mol> is a "
input_ids = tokenizer(text, return_tensors="pt").input_ids.cuda()
output = model.generate(input_ids, do_sample=False, max_new_tokens=200, num_beams=1, num_return_sequences=1)
print(tokenizer.decode(output[0]))

<s>The <mol> <m>C <m>C <m>( <m>= <m>O <m>) <m>O <m>c <m>1 <m>c <m>c <m>c <m>c <m>c <m>1 <m>C <m>( <m>= <m>O <m>) <m>O </mol> is a potent inhibitor of the human immunodeficiency virus type 1 reverse transcriptase. The <mol> <m>C <m>C <m>( <m>= <m>O <m>) <m>O <m>c <m>1 <m>c <m>c <m>c <m>c <m>c <m>1 <m>C <m>( <m>= <m>O <m>) <m>O </mol> (PA) is a potent inhibitor of the human immunodeficiency virus type 1 (HIV-1) reverse transcriptase (RT) in vitro. The inhibition is noncompetitive with respect to the template-primer and noncompetitive or uncompetitive with respect to the deoxynucleoside triphosphate. The inhibition is not reversed by dilution or by increasing the concentration of the template-primer. The inhibition is not reversed by increasing the concentration of the deoxynucleoside triphosphate. The inhibition is not reversed by increasing the concentration of the enzyme. The inhibition is


In [ ]:
text = "A material with two iron atoms and three oxygen atoms is <material>"
input_ids = tokenizer(text, return_tensors="pt").input_ids.cuda()
output = model.generate(input_ids, do_sample=True, max_new_tokens=200, num_beams=4, num_return_sequences=4)
for s in output:
    print(tokenizer.decode(s))

<s>A material with two iron atoms and three oxygen atoms is <material> undrum. A material with three iron atoms and two oxygen atoms is ferric oxide. A material with four iron atoms and one oxygen atom is ferrous oxide. A material with four iron atoms and two oxygen atoms is ferrous oxide. A material with four iron atoms and three oxygen atoms is ferrous oxide. A material with four iron atoms and four oxygen atoms is ferrous oxide. A material with four iron atoms and five oxygen atoms is ferrous oxide. A material with four iron atoms and six oxygen atoms is ferrous oxide. A material with four iron atoms and seven oxygen atoms is ferrous oxide. A material with four iron atoms and eight oxygen atoms is ferrous oxide. A material with four iron atoms and nine oxygen atoms is ferrous oxide. A material with four iron atoms and ten oxygen atoms is ferrous oxide.
<s>A material with two iron atoms and three oxygen atoms is <material> undrum. A material with three iron atoms and two oxygen atoms

In [ ]:
text = "The molecule is a steroid ester that is methyl (17E)-pregna-4,17-dien-21-oate substituted by oxo groups at positions 3 and 11. It is a 3-oxo-Delta(4) steroid, an 11-oxo steroid, a steroid ester and a methyl ester. It derives from a hydride of a pregnane. The SMILES is <mol>"
input_ids = tokenizer(text, return_tensors="pt").input_ids.cuda()
output = model.generate(input_ids, do_sample=True, top_p=0.9, max_new_tokens=200, num_beams=4, num_return_sequences=1)

print(tokenizer.decode(output[0]))

<s>The molecule is a steroid ester that is methyl (17E)-pregna-4,17-dien-21-oate substituted by oxo groups at positions 3 and 11. It is a 3-oxo-Delta(4) steroid, an 11-oxo steroid, a steroid ester and a methyl ester. It derives from a hydride of a pregnane. The SMILES is <mol> <m>C <m>C <m>[C@] <m>1 <m>( <m>O <m>) <m>C <m>C <m>[C@H] <m>2 <m>[C@@H] <m>3 <m>C <m>C <m>C <m>4 <m>= <m>C <m>C <m>( <m>= <m>O <m>) <m>C <m>C <m>[C@@H] <m>4 <m>[C@H] <m>3 <m>C <m>C <m>[C@@] <m>2 <m>1 <m>C </mol>  .</s>


In [ ]:
input_ids.shape

torch.Size([1, 112])

In [ ]:
output = model.generate(torch.cat([input_ids, input_ids], dim=0), do_sample=True, top_p=0.9, max_new_tokens=200, num_beams=4, num_return_sequences=1)


In [ ]:
print(tokenizer.decode(output[1]))

<s>The molecule is a steroid ester that is methyl (17E)-pregna-4,17-dien-21-oate substituted by oxo groups at positions 3 and 11. It is a 3-oxo-Delta(4) steroid, an 11-oxo steroid, a steroid ester and a methyl ester. It derives from a hydride of a pregnane. The SMILES is <mol> <m>C <m>C <m>[C@] <m>1 <m>( <m>O <m>) <m>C <m>C <m>[C@H] <m>2 <m>[C@@H] <m>3 <m>C <m>C <m>C <m>4 <m>= <m>C <m>C <m>( <m>= <m>O <m>) <m>C <m>C <m>[C@@H] <m>4 <m>[C@H] <m>3 <m>C <m>C <m>[C@@] <m>2 <m>1 <m>C </mol>  .</s>


In [ ]:
text_list = [
"a protein that folds fast",
"a transmembrane protein",
"a protein that transport protons across membranes",
"a protein that cut polypeptides",
"a protein that binds to Zinc ions",
"a protein that has 7 transmembrane helices",
"a protein that aborb photon energies",
"a protein emits fluorescence light",
"a protein that is an enzyme involves drug metabolism",
"a protein that transport protons across membranes",
]

with open('/home/shufxi/protein.txt', 'w') as f:
    for text in text_list:
        print(text)
        input_ids = tokenizer(text + ' <protein>', return_tensors="pt").input_ids.cuda()
        output = model.generate(input_ids, do_sample=True, top_p=0.95, max_new_tokens=1000, num_return_sequences=20)
        f.write(text + '\n')
        for s in output:
            pred = tokenizer.decode(s)
            if '</protein>' not in pred:
                continue
            pred_protein = pred[pred.find('<protein>') + len('<protein>'):pred.find('</protein>')]
            pred_protein = pred_protein.replace('<a>', '').replace(' ', '')
            f.write(pred_protein + '\n')
        f.write('\n')


a protein that folds fast
a transmembrane protein
a protein that transport protons across membranes
a protein that cut polypeptides
a protein that binds to Zinc ions
a protein that has 7 transmembrane helices
a protein that aborb photon energies
a protein emits fluorescence light
a protein that is an enzyme involves drug metabolism
a protein that transport protons across membranes


Bad pipe message: %s [b'\xfac\x0fn\xb9\x8b[\x80\xbb1\xfe\xd7Y\xc0E\xe3\xd1\x98 \x017T\x9bw\xf3\xbb\xe7\xe3\x1b\xb3\xcb\x86\x9dJ\xc4Y\x9c\xbdX5]C\xb6I\xa7\x87\xd1\x0c\xb9\xd8\xe2\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03', b'\x08\x07\x08\x08\x08', b'\n\x08\x0b\x08\x04\x08\x05\x08']
Bad pipe message: %s [b'\x01\x05\x01\x06\x01']
Bad pipe message: %s [b'4\xde\xd0w\xc7\x04\x9dy']
Bad pipe message: %s [b"4\xf3\x96\xa0h\xb6L\xfd\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14"]
Bad pipe message: %s [b'\x15:\xe6\xb7#\xeb)\x81\xb7%\xeab\x8f\xa9#\xf4\x05\x8c\x00\x00\xa6\xc0,

In [ ]:
input_ids = tokenizer('A short protein with only beta helix is <protein>', return_tensors="pt").input_ids.cuda()
output = model.generate(input_ids, do_sample=True, top_p=0.95, max_new_tokens=300, num_return_sequences=20)
for s in output:
    s = tokenizer.decode(s)
    if '</protein>' not in s:
        continue
    print(s)
    s = s[s.find('<protein>') + len('<protein>'):s.find('</protein>')]
    s = s.replace('<a>', '').replace(' ', '')
    print(s)
    print()

<s>A short protein with only beta helix is <protein> <a>M <a>S <a>L <a>Q <a>E <a>M <a>F <a>R <a>F <a>P <a>M <a>G <a>L <a>L <a>L <a>G <a>S <a>V <a>L <a>L <a>V <a>A <a>S <a>A <a>P <a>A <a>T <a>L <a>E <a>P <a>P <a>G <a>C <a>S <a>N <a>K <a>E <a>Q <a>Q <a>V <a>T <a>V <a>S <a>H <a>N <a>L <a>S <a>N <a>S <a>P <a>A <a>S <a>N <a>L <a>S <a>L <a>G <a>H <a>L <a>E <a>S <a>L <a>K <a>T <a>G <a>Q <a>D <a>R <a>M <a>L <a>V <a>S <a>S <a>D <a>G <a>K <a>S <a>V <a>Q <a>V <a>T <a>E <a>G <a>G <a>W <a>S <a>G <a>L <a>G <a>C <a>P </protein> inhibitory to HIV-1 integrase. The HIV-1 virus is the causative agent of the acquired immunodeficiency syndrome (AIDS). Integration of the viral DNA into the host chromosome is mediated by the viral proteins, the viral enzyme integrase and the viral DNA-terminal protein, termed 3' processing endonuclease. We studied the effect of a 16-mer peptide, a beta helix of the HIV-1 capsid protein p24, on both enzymes. In this peptide (p24(36)(25-40)), Ser42 was modified with a <mol> <m

In [12]:
input_ids = tokenizer('I will tell you something about SARS-COV-2 main protase.', return_tensors="pt").input_ids.cuda()
output = model.generate(input_ids, do_sample=True, top_p=0.95, max_new_tokens=300, num_return_sequences=4)
for s in output:
    s = tokenizer.decode(s)
    print(s)

<s>I will tell you something about SARS-COV-2 main protase. I will not show you its 3-D structure, because it is known. But the thing is that all the COVID-19 vaccines currently used in the world, are based on this enzyme structure. I want to tell you something very interesting about this enzyme. It is a single-chain 3C-like cysteine protease with a molecular weight of 294 kDa. It is an exogenous protein. It is coded in the RNA virus, and it is involved in viral gene processing, structural protein disassembly, and viral maturation. It cleaves the viral polyprotein into smaller proteins. It is used to disassemble the coronavirus and virus. This protein is a very useful molecule for SARS-CoV-2. It is very dangerous because it is a single-chain enzyme and is a part of the RNA virus. The antibody that you find in these vaccines will only affect the viral enzyme. I do not like the vaccines, but I will tell you the problem. I am trying to explain to you that it is possible to create another 

In [16]:
input_ids = tokenizer('A drug for SARS-CoV-2 is <mol>', return_tensors="pt").input_ids.cuda()
output = model.generate(input_ids, num_beams=4, max_new_tokens=300, num_return_sequences=4)
for s in output:
    s = tokenizer.decode(s)
    print(s)
    mol = s[s.find('<mol>') + len('<mol>'):s.find('</mol>')]
    mol = mol.replace('<m>', '').replace(' ', '')
    print(mol)

<s>A drug for SARS-CoV-2 is <mol> <m>C <m>c <m>1 <m>c <m>c <m>c <m>c <m>( <m>C <m>) <m>c <m>1 <m>O <m>C <m>C <m>( <m>= <m>O <m>) <m>N <m>[C@@H] <m>( <m>C <m>c <m>1 <m>c <m>c <m>c <m>c <m>c <m>1 <m>) <m>[C@@H] <m>( <m>O <m>) <m>C <m>[C@H] <m>( <m>C <m>c <m>1 <m>c <m>c <m>c <m>c <m>c <m>1 <m>) <m>N <m>C <m>( <m>= <m>O <m>) <m>[C@H] <m>( <m>C <m>( <m>C <m>) <m>C <m>) <m>N <m>1 <m>C <m>C <m>C <m>N <m>C <m>1 <m>= <m>O </mol> that inhibits the SARS-CoV-2 main protease. The SARS-CoV-2 main protease (Mpro) is a promising drug target for the treatment of COVID-19. In this study, we performed a virtual screening of Food and Drug Administration (FDA)-approved drugs against SARS-CoV-2 Mpro and identified lopinavir as a potent inhibitor of SARS-CoV-2 Mpro. Lopinavir inhibited SARS-CoV-2 Mpro with an IC50 value of 0.001 M, which is 1000-fold more potent than that of remdesivir, the only FDA-approved drug for the treatment of COVID-19. Lopinavir inhibited SARS-CoV-2 replication in Vero E6 cells with 